https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29

In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
# from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

In [2]:
wine = datasets.load_wine()
X,y = wine.data, wine.target.ravel() 
X.shape

(178, 13)

In [3]:
def generate_nan(Xtrain, missing_rate):
  Xshape = Xtrain.shape
  na_id = np.random.randint(0,Xtrain.size,round(missing_rate*Xtrain.size))
  Xtr_nan = Xtrain.flatten()
  Xtr_nan[na_id] = np.nan 
  return Xtr_nan.reshape(Xshape)

In [4]:
def one_iter(clf, clf1, clf2, missing_rate):
    X1, X2, y1, y2 = train_test_split(X,y.reshape((-1,1)),test_size=0.3)
    Xtrain1, Xtest1, ytrain1, ytest1 = train_test_split(X1,y1.reshape((-1,1)),test_size=0.5)
    Xtrain2, Xtest2, ytrain2, ytest2 = train_test_split(X2,y2.reshape((-1,1)),test_size=0.5)            

    Xtrain1_ori = Xtrain1[:,ndel1:].copy()
    Xtrain2_ori = Xtrain2[:,:ndel2].copy()
    Xtest1_ori = Xtest1[:,ndel1:].copy()
    Xtest2_ori = Xtest2[:,:ndel2].copy()    
    
    Xtrain1[:, :ndel1] = np.nan
    Xtest1[:,:ndel1] = np.nan
    Xtrain2[:, ndel2:] = np.nan
    Xtest2[:,ndel2:] = np.nan
    
    Xtrain1[:,ndel1:] = generate_nan(Xtrain1[:,ndel1:] , missing_rate)
    Xtest1[:,ndel1:] = generate_nan(Xtest1[:,ndel1:], missing_rate)
    Xtrain2[:,:ndel2] = generate_nan(Xtrain2[:,:ndel2], missing_rate)
    Xtest2[:,:ndel2] = generate_nan(Xtest2[:,:ndel2], missing_rate)
    
    Xtrain = np.vstack((Xtrain1, Xtrain2))
    Xtest = np.vstack((Xtest1, Xtest2))
    ytrain = np.vstack((ytrain1, ytrain2)) 
    ytest = np.vstack((ytest1, ytest2))
    
    Xtrain1 = Xtrain1[:,ndel1:]
    Xtrain2 = Xtrain2[:,:ndel2]
    Xtest1 = Xtest1[:,ndel1:]
    Xtest2 = Xtest2[:,:ndel2]
    
    scaler = MinMaxScaler()
    scaler.fit(Xtrain1)
    Xtrain1 = scaler.transform(Xtrain1)
    Xtrain1_ori = scaler.transform(Xtrain1_ori)
    Xtest1 = scaler.transform(Xtest1)
    Xtest1_ori = scaler.transform(Xtest1_ori)
    
    scaler.fit(Xtrain2)
    Xtrain2 = scaler.transform(Xtrain2)
    Xtrain2_ori = scaler.transform(Xtrain2_ori)
    Xtest2 = scaler.transform(Xtest2)    
    Xtest2_ori = scaler.transform(Xtest2_ori)    

    scaler.fit(Xtrain)
    Xtrain = scaler.transform(Xtrain)
    Xtest = scaler.transform(Xtest)
    Xtrain = SoftImpute(verbose = False).fit_transform(Xtrain)
    Xtest = SoftImpute(verbose = False).fit_transform(Xtest)
    Xtrain1 = SoftImpute(verbose = False).fit_transform(Xtrain1)
    Xtrain2 = SoftImpute(verbose = False).fit_transform(Xtrain2)
    Xtest1 = SoftImpute(verbose = False).fit_transform(Xtest1)
    Xtest2 = SoftImpute(verbose = False).fit_transform(Xtest2)
   
    clf.fit(Xtrain, ytrain)
    clf1.fit(Xtrain1, ytrain1)
    clf2.fit(Xtrain2, ytrain2)
        
    acc_res = np.array([np.mean(clf.predict(Xtest[:len(ytest1),:])==ytest1.flatten()), 
                     np.mean(clf.predict(Xtest[len(ytest1):,:])==ytest2.flatten()),
                    clf1.score(Xtest1, ytest1),  clf2.score(Xtest2, ytest2)])
    mse_impute = np.array([np.mean((Xtrain1 - Xtrain1_ori)**2), np.mean((Xtest1 - Xtest1_ori)**2),
                           np.mean((Xtrain[:len(Xtrain1),ndel1:] - Xtrain1_ori)**2), 
                           np.mean((Xtest[:len(Xtest1),ndel1:] - Xtest1_ori)**2),
                           np.mean((Xtrain2 - Xtrain2_ori)**2), np.mean((Xtest2 - Xtest2_ori)**2),
                           np.mean((Xtrain[len(Xtrain1):,:ndel2] - Xtrain2_ori)**2), 
                           np.mean((Xtest[len(Xtest1):,:ndel2] - Xtest2_ori)**2)]).reshape((-1,2))
    return [acc_res, mse_impute]    

In [5]:
ndel1, ndel2 = 2, 11
nruns = 1000
clf = LogisticRegression(random_state=0)
clf1 = LogisticRegression(random_state=0)
clf2 = LogisticRegression(random_state=0)
res = [one_iter(clf, clf1, clf2, missing_rate = .2) for i in range(nruns)]
acc_res = np.array([res[i][0] for i in range(nruns)])
np.mean(acc_res, axis = 0), np.var(acc_res, axis = 0) 

(array([0.90820968, 0.87474074, 0.9166129 , 0.85959259]),
 array([0.00135836, 0.00417569, 0.00127292, 0.00666848]))

In [6]:
mse_res = np.array([res[i][1] for i in range(nruns)])
np.mean(mse_res, axis = 0), np.var(mse_res, axis = 0) 

(array([[0.00793265, 0.00778459],
        [0.01020385, 0.01021916],
        [0.01347122, 0.01333388],
        [0.02273571, 0.02522764]]),
 array([[2.96438059e-06, 2.98049338e-06],
        [7.91013851e-06, 9.72243492e-06],
        [2.08678150e-05, 2.05860949e-05],
        [4.81097090e-05, 7.95876240e-05]]))

In [7]:
res = [one_iter(clf, clf1, clf2, missing_rate = .4) for i in range(nruns)]
acc_res = np.array([res[i][0] for i in range(nruns)])
print(np.mean(acc_res, axis = 0), np.var(acc_res, axis = 0))
mse_res = np.array([res[i][1] for i in range(nruns)])
np.mean(mse_res, axis = 0), np.var(mse_res, axis = 0) 

[0.87737097 0.83837037 0.88356452 0.8082963 ] [0.00170067 0.00553841 0.00170631 0.00824557]


(array([[0.01816288, 0.017888  ],
        [0.02054043, 0.02052362],
        [0.03310652, 0.03225823],
        [0.04181108, 0.0445817 ]]),
 array([[1.05895279e-05, 1.11543723e-05],
        [2.00408793e-05, 2.18754290e-05],
        [8.90937313e-05, 8.27346855e-05],
        [1.26182175e-04, 1.86147274e-04]]))

In [8]:
res = [one_iter(clf, clf1, clf2, missing_rate = .6) for i in range(nruns)]
acc_res = np.array([res[i][0] for i in range(nruns)])
print(np.mean(acc_res, axis = 0), np.var(acc_res, axis = 0))
mse_res = np.array([res[i][1] for i in range(nruns)])
np.mean(mse_res, axis = 0), np.var(mse_res, axis = 0) 

[0.83112903 0.78288889 0.83464516 0.72988889] [0.00265689 0.00625097 0.00270512 0.01036098]


(array([[0.03206217, 0.03213652],
        [0.03459798, 0.03493264],
        [0.06281139, 0.06051823],
        [0.06876991, 0.07085559]]),
 array([[2.97356333e-05, 3.45679728e-05],
        [4.14469527e-05, 4.80535432e-05],
        [2.98017306e-04, 2.46721724e-04],
        [3.23755436e-04, 3.73294097e-04]]))

In [9]:
res = [one_iter(clf, clf1, clf2, missing_rate = .8) for i in range(nruns)]
acc_res = np.array([res[i][0] for i in range(nruns)])
print(np.mean(acc_res, axis = 0), np.var(acc_res, axis = 0))
mse_res = np.array([res[i][1] for i in range(nruns)])
np.mean(mse_res, axis = 0), np.var(mse_res, axis = 0) 

[0.76154839 0.71266667 0.76072581 0.64496296] [0.00339473 0.00827357 0.00359203 0.01173197]


(array([[0.0558572 , 0.05451565],
        [0.05820464, 0.05716611],
        [0.11443691, 0.10953645],
        [0.11536247, 0.117818  ]]),
 array([[9.23300904e-05, 8.66914640e-05],
        [1.08458160e-04, 9.91440606e-05],
        [3.43350158e-03, 7.39546008e-03],
        [3.58431275e-03, 8.37882502e-03]]))

## SVC

In [10]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf1 = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf2 = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [11]:
res = [one_iter(clf, clf1, clf2, missing_rate = .2) for i in range(nruns)]
acc_res = np.array([res[i][0] for i in range(nruns)])
print(np.mean(acc_res, axis = 0), np.var(acc_res, axis = 0))
mse_res = np.array([res[i][1] for i in range(nruns)])
np.mean(mse_res, axis = 0), np.var(mse_res, axis = 0) 

[0.93845161 0.89525926 0.93908065 0.88151852] [0.00084499 0.003504   0.00087806 0.00595254]


(array([[0.00796429, 0.00778817],
        [0.01035619, 0.01037752],
        [0.01361895, 0.01302472],
        [0.02273803, 0.02455241]]),
 array([[2.91588789e-06, 3.06120260e-06],
        [8.61054041e-06, 1.05720650e-05],
        [2.04653522e-05, 1.88578741e-05],
        [4.33736773e-05, 6.81037288e-05]]))

In [12]:
res = [one_iter(clf, clf1, clf2, missing_rate = .4) for i in range(nruns)]
acc_res = np.array([res[i][0] for i in range(nruns)])
print(np.mean(acc_res, axis = 0), np.var(acc_res, axis = 0))
mse_res = np.array([res[i][1] for i in range(nruns)])
np.mean(mse_res, axis = 0), np.var(mse_res, axis = 0) 

[0.90206452 0.84618519 0.90056452 0.81725926] [0.00138002 0.00541233 0.00140473 0.00884725]


(array([[0.01832069, 0.01808816],
        [0.02088356, 0.02090323],
        [0.03359368, 0.03183512],
        [0.04173581, 0.04343481]]),
 array([[1.07625205e-05, 1.20191620e-05],
        [2.17472221e-05, 2.38887889e-05],
        [1.00860028e-04, 8.09709682e-05],
        [1.39867421e-04, 1.76458672e-04]]))

In [13]:
res = [one_iter(clf, clf1, clf2, missing_rate = .6) for i in range(nruns)]
acc_res = np.array([res[i][0] for i in range(nruns)])
print(np.mean(acc_res, axis = 0), np.var(acc_res, axis = 0))
mse_res = np.array([res[i][1] for i in range(nruns)])
np.mean(mse_res, axis = 0), np.var(mse_res, axis = 0) 

[0.84554839 0.77844444 0.84187097 0.72222222] [0.00226957 0.00759626 0.00231571 0.01299588]


(array([[0.03257797, 0.03232201],
        [0.03510129, 0.0350439 ],
        [0.06233989, 0.06050585],
        [0.06784773, 0.07056643]]),
 array([[2.90341699e-05, 3.44646520e-05],
        [3.94262675e-05, 4.52978710e-05],
        [3.11100435e-04, 2.74979537e-04],
        [3.51280210e-04, 4.30425196e-04]]))

In [14]:
res = [one_iter(clf, clf1, clf2, missing_rate = .8) for i in range(nruns)]
acc_res = np.array([res[i][0] for i in range(nruns)])
print(np.mean(acc_res, axis = 0), np.var(acc_res, axis = 0))
mse_res = np.array([res[i][1] for i in range(nruns)])
np.mean(mse_res, axis = 0), np.var(mse_res, axis = 0) 

[0.76716129 0.68566667 0.76222581 0.59448148] [0.00344482 0.00928646 0.00402483 0.01407188]


(array([[0.05589455, 0.05504481],
        [0.05826093, 0.05771055],
        [0.11106524, 0.1045343 ],
        [0.11164699, 0.11258128]]),
 array([[9.58446104e-05, 9.39613723e-05],
        [1.14575916e-04, 1.16135100e-04],
        [1.90441733e-03, 9.11700185e-04],
        [1.95785018e-03, 1.32027019e-03]]))